In [1]:
import pandas as pd

submission = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')

In [2]:
submission

,id,class,predictionstring
0,18409261F5C2,NaN,NaN
1,D46BCB48440A,NaN,NaN
2,0FB0700DAF44,NaN,NaN
3,D72CB1C11673,NaN,NaN
4,DF920E0A7337,NaN,NaN


In [3]:
from tqdm import tqdm
import os
test_names, test_texts = [], []
test_texts_list = []
for f in tqdm(list(os.listdir('../input/feedback-prize-2021/test'))):
    test_names.append(f.replace('.txt', ''))
    test_texts.append(open('../input/feedback-prize-2021/test/' + f, 'r', encoding='utf-8').read())
for t in test_texts:
    test_texts_list.append(t.split())
test_text_df = pd.DataFrame({'id': test_names, 'text': test_texts, 'text_list': test_texts_list})

100%|██████████| 5/5 [00:00<00:00, 3819.95it/s]


In [4]:
test_text_df

,id,text,text_list
0,D46BCB48440A,"When people ask for advice,they sometimes talk...","[When, people, ask, for, advice,they, sometime..."
1,DF920E0A7337,Have you ever asked more than one person for h...,"[Have, you, ever, asked, more, than, one, pers..."
2,0FB0700DAF44,"During a group project, have you ever asked a ...","[During, a, group, project,, have, you, ever, ..."
3,D72CB1C11673,Making choices in life can be very difficult. ...,"[Making, choices, in, life, can, be, very, dif..."
4,18409261F5C2,80% of Americans believe seeking multiple opin...,"[80%, of, Americans, believe, seeking, multipl..."


In [5]:
from datasets import Dataset

test_datasets = Dataset.from_pandas(test_text_df)

In [6]:
test_datasets

Dataset({
    features: ['id', 'text', 'text_list'],
    num_rows: 5
})

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096', add_prefix_space=True)

def preparing_test_dataset(examples):
    encoding = tokenizer(examples['text_list'], truncation=True, padding=False, max_length = 2048, is_split_into_words=True)
    total= len(encoding['input_ids'])
    encoding['word_ids']=[]
    encoding['id']=[]
    for i in range(total):
        word_idx = encoding.word_ids(batch_index=i)
        encoding['word_ids'].append(word_idx)
        encoding['id'].append(examples['id'][i])
    return encoding

tokenized_test_datasets = test_datasets.map(preparing_test_dataset, batched=True, batch_size=5, remove_columns=test_datasets.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
from transformers import AutoModelForTokenClassification, AutoConfig, Trainer, DataCollatorForTokenClassification
import numpy as np
data_collator = DataCollatorForTokenClassification(tokenizer)
all_predictions = []
for fold in range(5):
    model_path = './best_model/longformer-baseline_fold'+str(fold)+'/'
    config = AutoConfig.from_pretrained(model_path)
    model = AutoModelForTokenClassification.from_pretrained(model_path, config = config)
    
    trainer = Trainer(
        model = model,
        train_dataset=None,
        eval_dataset=tokenized_test_datasets,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )
    predictions, _, _ = trainer.predict(test_dataset = tokenized_test_datasets)
    print("shape of prediciton", predictions.shape)
    all_predictions.append(predictions.astype(np.float32))

The following columns in the test set  don't have a corresponding argument in `LongformerForTokenClassification.forward` and have been ignored: word_ids, id.
***** Running Prediction *****
  Num examples = 5
  Batch size = 8
Input ids are automatically padded from 1292 to 1536 to be a multiple of `config.attention_window`: 512


loading configuration file ./best_model/longformer-baseline_fold1/config.json
Model config LongformerConfig {
  "_name_or_path": "./best_model/longformer-baseline_fold1/",
  "architectures": [
    "LongformerForTokenClassification"
  ],
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14"
  },
  "ignore_attention_mask": false,
  "init

shape of prediciton (5, 1292, 15)


All model checkpoint weights were used when initializing LongformerForTokenClassification.

All the weights of LongformerForTokenClassification were initialized from the model checkpoint at ./best_model/longformer-baseline_fold1/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LongformerForTokenClassification for predictions without further training.
No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set  don't have a corresponding argument in `LongformerForTokenClassification.forward` and have been ignored: word_ids, id.
***** Running Prediction *****
  Num examples = 5
  Batch size = 8
Input ids 

loading configuration file ./best_model/longformer-baseline_fold2/config.json
Model config LongformerConfig {
  "_name_or_path": "./best_model/longformer-baseline_fold2/",
  "architectures": [
    "LongformerForTokenClassification"
  ],
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14"
  },
  "ignore_attention_mask": false,
  "init

...

All model checkpoint weights were used when initializing LongformerForTokenClassification.

All the weights of LongformerForTokenClassification were initialized from the model checkpoint at ./best_model/longformer-baseline_fold3/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LongformerForTokenClassification for predictions without further training.
No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set  don't have a corresponding argument in `LongformerForTokenClassification.forward` and have been ignored: word_ids, id.
***** Running Prediction *****
  Num examples = 5
  Batch size = 8
Input ids 

loading configuration file ./best_model/longformer-baseline_fold4/config.json
Model config LongformerConfig {
  "_name_or_path": "./best_model/longformer-baseline_fold4/",
  "architectures": [
    "LongformerForTokenClassification"
  ],
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14"
  },
  "ignore_attention_mask": false,
  "init

shape of prediciton (5, 1292, 15)


All model checkpoint weights were used when initializing LongformerForTokenClassification.

All the weights of LongformerForTokenClassification were initialized from the model checkpoint at ./best_model/longformer-baseline_fold4/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LongformerForTokenClassification for predictions without further training.
No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set  don't have a corresponding argument in `LongformerForTokenClassification.forward` and have been ignored: word_ids, id.
***** Running Prediction *****
  Num examples = 5
  Batch size = 8
Input ids 

shape of prediciton (5, 1292, 15)


In [9]:
raw_preds = 0
for preds in all_predictions:
    preds = preds/5
    raw_preds+=preds

In [23]:
import torch
x = torch.nn.Softmax(dim=-1)
input = torch.tensor(raw_preds)
output = x(input)
raw_preds=output.numpy()

In [24]:
from collections import defaultdict

def label_dict():
    train = pd.read_csv('../input/feedback-prize-2021/train.csv')
    classes = train.discourse_type.unique().tolist()
    tags = defaultdict()
    for i, c in enumerate(classes):
        tags[f'B-{c}'] = i
        tags[f'I-{c}'] = i + len(classes)
    tags[f'O'] = len(classes) * 2
    tags[f'Special'] = -100
    l2i = dict(tags)
    i2l = defaultdict()
    for k, v in l2i.items(): 
        i2l[v] = k
    i2l[-100] = 'Special'
    i2l = dict(i2l)
    N_LABELS = len(i2l) - 1 # not accounting for -100
    return i2l, l2i, N_LABELS

In [25]:
def link_evidence(oof):
  if not len(oof):
    return oof
  
  def jn(pst, start, end):
    return " ".join([str(x) for x in pst[start:end]])
  
  thresh = 1
  idu = oof['id'].unique()
  eoof = oof[oof['class'] == "Evidence"]
  neoof = oof[oof['class'] != "Evidence"]
  eoof.index = eoof[['id', 'class']]
  for thresh2 in range(26, 27, 1):
    retval = []
    for idv in tqdm(idu, desc='link_evidence', leave=False):
      for c in ['Evidence']:
        q = eoof[(eoof['id'] == idv)]
        if len(q) == 0:
          continue
        pst = []
        for r in q.itertuples():
          pst = [*pst, -1,  *[int(x) for x in r.predictionstring.split()]]
        start = 1
        end = 1
        for i in range(2, len(pst)):
          cur = pst[i]
          end = i
          if  ((cur == -1) and ((pst[i + 1] > pst[end - 1] + thresh) or (pst[i + 1] - pst[start] > thresh2))):
            retval.append((idv, c, jn(pst, start, end)))
            start = i + 1
        v = (idv, c, jn(pst, start, end + 1))
        retval.append(v)
    roof = pd.DataFrame(retval, columns=['id', 'class', 'predictionstring'])
    roof = roof.merge(neoof, how='outer')
    return roof

In [52]:
proba_thresh = {
    "Lead": 0.7,
    "Position": 0.55,
    "Evidence": 0.65,
    "Claim": 0.55,
    "Concluding Statement": 0.7,
    "Counterclaim": 0.5,
    "Rebuttal": 0.55,
}
#discourse length threshold
min_thresh = {
    "Lead": 9,
    "Position": 5,
    "Evidence": 14,
    "Claim": 3,
    "Concluding Statement": 11,
    "Counterclaim": 6,
    "Rebuttal": 4,
}

def postprocess_fb_predictions(
    eval_examples,
    eval_datasets,
    predictions,
):
    print(predictions.shape)
    preds = np.argmax(predictions, axis=-1)
    # pred_score = np.max(predictions, axis=-1)
    pred_score = predictions
    i2l, l2i, N_LABELS = label_dict()
    all_prediction = []
    all_pred_score=[]
    for k, (label_pred, label_pred_score) in tqdm(enumerate(zip(preds, pred_score))):
      token_preds = [i2l[i] for i in label_pred]
      each_prediciton = []
      each_prediction_score = []
      word_ids = eval_datasets['word_ids'][k]
      previous_word_idx = -1
      previous_word_label = -100
      word_prediction_score=[]
      for idx, word_idx in enumerate(word_ids):
        if word_idx == None:
          continue
        elif word_idx != previous_word_idx:
          each_prediciton.append(token_preds[idx])
          previous_word_idx = word_idx
          if len(word_prediction_score)!=0:
            each_prediction_score.append(word_prediction_score)
          word_prediction_score=[]
          word_prediction_score.append(label_pred_score[idx][label_pred[idx]])
          previous_word_label = label_pred[idx]
        else:
          word_prediction_score.append(label_pred_score[idx][previous_word_label])
      each_prediction_score.append(word_prediction_score)
      all_prediction.append(each_prediciton)
      all_pred_score.append(each_prediction_score)
    final_pred = []
    for i in range(len(eval_datasets)):
      idx = eval_datasets['id'][i]
      pred = all_prediction[i]
      pred_score = all_pred_score[i]
      j=0
      while j < len(pred):
        cls = pred[j]
        if cls =='O': 
            j+=1
        else: 
            cls = cls.replace('B', 'I')
        end = j+1
        while end < len(pred) and pred[end] == cls:
            end +=1
        final_pred_score = []
        for item in pred_score[j:end]:
          final_pred_score.extend(item)
        if cls != 'O' and cls!='' and sum(final_pred_score)/len(final_pred_score)>=proba_thresh[cls.replace('I-', '')] and end-j>=min_thresh[cls.replace('I-', '')]:
            final_pred.append((idx, cls.replace('I-', ''), ' '.join(map(str, list(range(j, end))))))
        j = end
    oof = pd.DataFrame(final_pred)
    oof.columns = ['id', 'class', 'predictionstring']
    
    oof = link_evidence(oof)
    return oof

In [53]:
oof = postprocess_fb_predictions(eval_examples=None, eval_datasets=tokenized_test_datasets, predictions=raw_preds)

(5, 1292, 15)


5it [00:00, 226.36it/s]


In [54]:
oof

,id,class,predictionstring
0,D46BCB48440A,Evidence,56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 7...
1,D46BCB48440A,Evidence,150 151 152 153 154 155 156 157 158 159 160 16...
2,D46BCB48440A,Evidence,223 224 225 226 227 228 229 230 231 232 233 23...
3,DF920E0A7337,Evidence,118 119 120 121 122 123 124 125 126 127 128 12...
4,DF920E0A7337,Evidence,310 311 312 313 314 315 316 317 318 319 320 32...
5,DF920E0A7337,Evidence,466 467 468 469 470 471 472 473 474 475 476 47...
6,0FB0700DAF44,Evidence,120 121 122 123 124 125 126 127 128 129 130 13...
7,0FB0700DAF44,Evidence,341 342 343 344 345 346 347 348 349 350 351 35...
8,D72CB1C11673,Evidence,94 95 96 97 98 99 100 101 102 103 104 105 106 ...
9,D72CB1C11673,Evidence,183 184 185 186 187 188 189 190 191 192 193 19...


In [55]:
oof.to_csv("submission2.csv", index=False)